In [ ]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import _mysql
import pandas as pd
import pylab as plt
import numpy as np
import random as rd
import itertools as itertools

# Plotting packages
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
from bokeh.charts import Bar, output_file, show
from bokeh.sampledata.autompg import autompg as df
%matplotlib inline
output_notebook()

# kslearn packages
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score

# smoothing 
from scipy.signal import savgol_filter

# regular expression
import re

mysqlFilePath = 'mysql://root:@localhost/clientsuccess?charset=utf8&use_unicode=0'

def replace_special(char):
    try:
        if char.encode('string-escape') == r'\x01':
            return True
        elif char.encode('string-escape') == r'\x00':
            return False
        else:
            return char
    except:
        return char

In [ ]:
"""
This function prints and plots the confusion matrix.
Normalization can be applied by setting `normalize=True`.
"""
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
engine = create_engine(mysqlFilePath, pool_recycle=3600)
connection = engine.connect()

In [ ]:
db_name = "kim_large_churn_tenants1"
tb_res =  pd.read_sql_query("""
    SELECT 
        *
    FROM 
        %s
        """%db_name, connection).applymap(lambda x: replace_special(x))

### Random Forest 

In [ ]:
def RF(features, Xtrain, ytrain, test_size = 0.3, n_estimators = 10, max_depth = 3, class_weight = None):

    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth = max_depth, class_weight = class_weight)
    clf = clf.fit(X_train, y_train)
    coeff = clf.get_params()

    print coeff

    print("score: %.4f" % clf.score(X_test, y_test))
    
    return clf

### Feature vector construction (non-normalized)

In [ ]:
def data_preprocessing(df):
    
    # tenant company features
    lst_tenants = df['tenant_id'].unique()
    
    for x in lst_tenants:
        name = "tenant"+str(x)
        mask = (df['tenant_id'] == x)
        mask = [1 if x == True else 0 for x in mask]
        df[name] = pd.Series(mask, index = df.index)

    tenant_names = ["tenant"+str(x) for x in lst_tenants]
    
    # including the other features.
    feature_lst = ['client_note_total_count', 'sub_duration', 'last_sub_duration', 'amount_per_day', 'sentiment']\
        + tenant_names
    
    X_original = df[feature_lst].as_matrix()
    y = np.array(df['churned'])
    return X_original, y , feature_lst

### Confusion Matrix

In [ ]:
def confusion_table_plot(y_test, y_pred):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()

    class_names = ['Non-Churn','Churn']
    plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized confusion matrix')

    plt.show()

In [ ]:
X_original, y, feature_lst = data_preprocessing(tb_res)
scaler = StandardScaler()
X_raw = scaler.fit_transform(X_original)
X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=.3, random_state = 70)

clf = RF(feature_lst, X_train, y_train, n_estimators = 1000, max_depth = 10, class_weight="auto")
y_pred = clf.predict(X_test)
confusion_table_plot(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred) 

### Recall was improved significantly. Now it is 86%.

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

### Feature importance scores were computed for companies.

In [ ]:
def plot_importance(feature_importance_lst, feature_lst):
    p=figure(plot_width=800, plot_height=200)  
    df = pd.DataFrame()
    df['importance'] = pd.Series(feature_importance_lst , index = feature_lst)
    p = Bar(df, values='importance',title="Feature Importance", legend=False,\
            plot_width=800, plot_height=300)
    p.xaxis.axis_label = 'Features'
    p.yaxis.axis_label = 'Importance'
    show(p)
                    
plot_importance(clf.feature_importances_, feature_lst)

### Precision-recall curve was improved significantly when compared with the case that all the tenant companies were considered. 

In [ ]:
def plot_precision_recall_curve(clf, X_test, y_test):
    y_score = clf.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_score)
    plt.plot(recall, precision, lw = 1, color='navy',label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title("Precision-Recall: Random Forest")
    plt.show()

In [ ]:
plot_precision_recall_curve(clf, X_test, y_test)

In [ ]:
average_precision_score(y_test, y_pred)

### ROC

In [ ]:
def plot_ROC(clf, X_test, y_test):
    y_score = clf.predict_proba(X_test)[:, 1]
    fpr_rt, tpr_rt, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr_rt, tpr_rt)

    plt.figure()
    lw = 2
    plt.plot(fpr_rt, tpr_rt, color='darkorange',lw=lw, label='Random Forest (area = %0.2f)' % roc_auc)

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()
plot_ROC(clf, X_test, y_test)

### For all tenant companies, predicted and true churn numbers are compared.

In [ ]:
lst_cl =[]
lst_true = []
lst_pred = []
lst_corr = []
lst_tenants = tb_res['tenant_id'].unique()

tb_res['churned_pred'] = pd.Series(clf.predict(X_raw), index = tb_res.index)
#tb_res['churned_pred'] = pd.Series(clf_reduced.predict(X), index = tb_res.index)
for tid in lst_tenants:
    df_t = tb_res[tb_res['tenant_id'] == tid]
    num_cl = len(df_t)
    num_true_churn = df_t['churned'].sum()
    num_pred_churn = df_t['churned_pred'].sum()
    num_corr_churn = (df_t['churned'] & df_t['churned_pred']).sum()
    #print num_cl, "\t", num_true_churn, "\t", num_pred_churn, "\t", num_corr_churn
    lst_cl.append(num_cl)
    lst_true.append(num_true_churn)
    lst_pred.append(num_pred_churn)
    lst_corr.append(num_corr_churn)
    
df_churn = pd.DataFrame(index = lst_tenants)
df_churn['total_clients'] = lst_cl
df_churn['true_churn'] = lst_true
df_churn['pred_churn'] = lst_pred
df_churn['correctly_pred_churn'] = lst_corr

In [ ]:
df_churn['churn_rate'] = df_churn['true_churn']/df_churn['total_clients']

### For the tenant ID 173 and 132, recall was now significantly better. When considering all tenants together, the random forest classifier could not detect any churn cases for the two tenant companies. Now, the RF classifier can do much better.  

In [ ]:
df_churn

### With this every encouraging results, I computed all the probability for renewal for individual customers and for different number of email communications. 

In [ ]:
# plot_cus_prob make a plot for renewl probability for a given customer feature vector (normalized).
# x_axis of the plot is normalized frequency. It needs to be converted back to the "unnormalized" frequency.

def plot_cus_prob(clf, x_norm, x_original):
    #lst = np.array(range(-15,16))/15.
    #lst = [np.power(3, x) for x in lst]
    lst = np.array(range(300))/300.*(X_train[:,0].max()-X_train[:,0].min()) + X_train[:,0].min()
    #print lst
    Xinit = x_norm
    f0 = []
    for alpha in lst:
        perturb = np.array(([1,0] + [0]*(len(Xinit)-2)))*alpha
        x = np.array([0, Xinit[1]] + list(Xinit[2:]))
        f0.append(x + perturb)
    f0= np.array(f0)

    y_score = clf.predict_proba(f0)[:, 0]

    p=figure(plot_width=400, plot_height=200, x_range = Range1d(X_train[:,1].min(),X_train[:,1].max() ),\
            y_range = Range1d(start=0, end=100))
    
    
    
    p.line(f0[:,0], y_score*100, line_width = 4, alpha=0.4)
    p.xaxis.axis_label = 'Number of communications per year'
    p.yaxis.axis_label = 'Satisfaction level'
    p.circle(Xinit[0], clf.predict_proba([Xinit])[:, 0]*100, size = 10, color = "purple", alpha = 0.7)
    
    # filter window length = 13 
    # degree 2 polynomial function used for fitting.
    y_score_smoothed = savgol_filter(y_score, 21, 2)
    p.circle(f0[:,0], y_score_smoothed*100)
    show(p)
    
    freq = x_original[0]
    freq1 = x_original[1]
    p = y_score_smoothed[0]
    p1 = y_score_smoothed[1]
    delp = p - p1
    delfreq = freq-freq1
    # print delfreq/freq
    
    elasticity = (freq * delp)/(delfreq * p)
    return elasticity
    

In [ ]:
# list of tenants
# I chose tenant id = 147
index_lst = tb_res[tb_res['tenant_id']==147].index
# 2 clients are considered.
index_lst = index_lst[48:50]
x_originals = [X_original[item] for item in index_lst]
x_norms = scaler.transform(x_originals)

lst = []
for x_norm, x_original in zip(x_norms, x_originals):
    lst.append(plot_cus_prob(clf, x_norm, x_original))